# Import

In [1]:
from huggingface_hub import login
login()

In [2]:
from datasets import load_dataset
import datasets
import random

# dataset load

In [3]:
DATASET_NAME = "OpenAssistant/oasst2"


In [4]:
ds = load_dataset(DATASET_NAME)["train"]
def filter_en_only(ex):
    return ex["lang"] == "en"

ds = ds.filter(filter_en_only)

# Build message_id -> example lookup
by_id = {ex["message_id"]: ex for ex in ds}

def is_single_turn_assistant(ex):
    if ex["role"] != "assistant":
        return False
    if ex["parent_id"] is None or ex["parent_id"] not in by_id:
        return False

    parent = by_id[ex["parent_id"]]

    return (
        parent["role"] == "prompter"
        and ex["lang"] == "en"
        and ex["review_result"] is True
        and (ex["rank"] == 0 or ex["rank"] is None)
    )

filtered = ds.filter(is_single_turn_assistant)

def to_sft(ex):
    parent = by_id[ex["parent_id"]]
    return {
        "instruction": parent["text"],
        "response": ex["text"]
    }

single_turn_sft = filtered.map(
    to_sft,
    remove_columns=filtered.column_names
)


In [5]:
single_turn_sft

Dataset({
    features: ['instruction', 'response'],
    num_rows: 14440
})

In [6]:
single_turn_sft_sampled = single_turn_sft.shuffle(seed=177013).select(range(1200))

In [7]:
single_turn_sft_sampled.to_json(
    "/Users/sirrea/workspaces/data_science/saltLM/data/oasst2_single_turn_sft_1200_sampled.jsonl",
    orient="records",
    lines=True
)

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

1435738